In [1]:
import cdsapi
import xarray as xr
import numpy as np

import os
import requests
requests.packages.urllib3.disable_warnings(
    requests.packages.urllib3.exceptions.InsecureRequestWarning)

In [2]:
os.environ['URL'] = 'https://ads.atmosphere.copernicus.eu/api/v2'
os.environ['KEY'] = '<uid>:<api-key>'

In [3]:
url = os.environ.get('URL')
key = os.environ.get('KEY')
client = cdsapi.Client(url=url, key=key)

In [4]:
client.retrieve(
    'cams-global-atmospheric-composition-forecasts',
    {
        'date' : '2021-07-19/2021-07-19',
        'type' : 'forecast',
        'format' : 'netcdf',
        'variable' : 'particulate_matter_2.5um',
        'time' : '12:00',
        'leadtime_hour' : [
            '0', '1', '2', '3', '4', '5',
            '6', '7', '8', '9', '10', '11',
            '12', '13', '14', '15', '16', '17',
            '18', '19', '20', '21', '22', '23'
        ],
        'area' : [15, 90, -15, 150],
    },
    'download.nc'
)

2021-07-20 14:16:12,259 INFO Welcome to the CDS
2021-07-20 14:16:12,261 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-atmospheric-composition-forecasts
2021-07-20 14:16:12,686 INFO Request is completed
2021-07-20 14:16:12,688 INFO Downloading https://download-0002.copernicus-atmosphere.eu/cache-compute-0002/cache/data4/adaptor.mars_constrained.internal-1626762080.6360798-19752-2-2d860404-56ae-47a3-82e7-8493a82c1f8d.nc to download.nc (540.1K)
2021-07-20 14:16:18,317 INFO Download rate 96K/s


Result(content_length=553052,content_type=application/x-netcdf,location=https://download-0002.copernicus-atmosphere.eu/cache-compute-0002/cache/data4/adaptor.mars_constrained.internal-1626762080.6360798-19752-2-2d860404-56ae-47a3-82e7-8493a82c1f8d.nc)

In [5]:
ds = xr.open_dataset('download.nc')

In [6]:
ds

<xarray.Dataset>
Dimensions:    (latitude: 76, longitude: 151, time: 24)
Coordinates:
  * longitude  (longitude) float32 90.0 90.4 90.8 91.2 ... 149.2 149.6 150.0
  * latitude   (latitude) float32 15.0 14.6 14.2 13.8 ... -14.2 -14.6 -15.0
  * time       (time) datetime64[ns] 2021-07-19T12:00:00 ... 2021-07-20T11:00:00
Data variables:
    pm2p5      (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-07-20 06:21:20 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...